In [8]:
import numpy as np
from scipy.constants import hbar, h, elementary_charge, pi

In [36]:
phi0 = h/(2*elementary_charge)
phi0

2.067833831170082e-15

In [ ]:
class unitConverter:
    
    def __init__(self, iu=)

In [ ]:
pparams = dict{
    iu = 5e-6,
    jc = 1e-6,
    cc = 40e-15,
    betac = 1
}

In [171]:
(2 * elementary_charge / hbar) * j1.Ir * (j1.Rr**2) * j1.Cr

2.0000000000000004

In [177]:
j1.j

2

In [ ]:
class unitConverter:
    
    def __init__(self,)

In [396]:
class JJ:
    
    def __init__(self, j = 2, iu = 5e-6, jc=1e-6, cc=40e-15, betac=1, sheetR=2, w=2):
        
        self.j = j
        self.iu = iu
        self.jc = jc
        self.cc = cc
        self.betac = betac
        
        # Physical params
        self.sheetR = sheetR # Sheet Resistance (Ohm/square)
        self.w = w           # Trace Width (um)
        
    @property
    def area(self):
        return self.j * self.iu / self.jc
    
    @property
    def Ir(self):
        return self.j * self.iu
    
    @property 
    def Cr(self):
        return self.area * self.cc
    
    @property
    def Rr(self):
        return np.sqrt(hbar*self.betac)/np.sqrt(2*elementary_charge*self.jc*self.cc) / self.area
    
    @property
    def RLen(self):
        return self.Rr * self.w / self.sheetR
    
    def __str__(self):
        return "Ic={:.3f}uA, d={:.3f}um, R={:.3f}Ohm, RLen={:.3f}um, C={:.3f}fF".format(self.Ir*1e6, 2*np.sqrt(self.area/pi), self.Rr, self.RLen, self.Cr*1e15)
    
    def __repr__(self):
        return "JJ:\n\t{}".format(str(self))

In [397]:
j1 = JJ(j=2, betac=1, w=2)
j1

JJ:
	Ic=10.000uA, d=3.568um, R=9.071Ohm, RLen=9.071um, C=400.000fF

In [398]:
j1.RLen

9.0706391136921862

In [173]:
j1.Ir

1e-05

In [174]:
j1.Rr

9.0706391136921862

In [175]:
j1.Cr

4.0000000000000006e-13

In [176]:
j1.area

10.000000000000002

# Standard JTL


PSCAN Units
- L = 1.5
- J = 2
- I = 3


In [379]:
class JTL:
    
    def __init__(self, iu = 5, jc = 1, cc = 40.5, betac = 1):
        
        ## Circuit params
        self.iu = iu * 1e-6  # A
        self.Jp = 2
        self.Lp = 1.5
        self.Ip = 3
        self.betac = betac
        
        ## Physical params
        self.sheetR = 2      # sheet resistance:          2Ohm/square
        self.w  = 1          # trace width :               1um
        self.Ll = 0.73e-12   # M2 Microstrip Inductance:  73pH/um at w=1um
        self.jc = jc * 1e-6  # Critical Current Density:     A/um^2
        self.cc = cc * 1e-15 # Junction Capacitance Density: F/um^2
        
        ## Netlist params
        self.jjmodel = "jjs"
    
    @property
    def vu(self):
        return np.sqrt((phi0 * self.jc) / (2*pi*self.cc))
    
    @property
    def lu(self):
        return (phi0/(2*pi)) / self.iu
    
    @property
    def ru(self):
        return self.vu/self.iu
    
    @property
    def Jr(self):
        return self.Jp * self.iu
    
    @property
    def Lr(self):
        return self.Lp * self.lu
    
    @property
    def Ir(self):
        return self.Ip * self.iu
    
    @property
    def Cr(self):
        return self.cc * self.Jr / self.jc
    
    @property
    def Rp(self):
        
        pass
    
    @property
    def Rr(self):
        
        
        # Calculate from betaC
        return np.sqrt((2*pi*self.betac)/(phi0 * self.Jr * self.Cr))
    
    @property
    def J1(self):
        return JJ(j=self.Jp, iu=self.iu, jc=self.jc, cc=self.cc, betac=self.betac)
    
    @property
    def LLen(self):
        return self.Lr / self.Ll
    
    ###############
    ### Methods ###
    ###############
    def netlist(self):
    #         .subckt JTL IN OUT
    #         B0 4 0 6 jjs area=4
    #         B1 5 0 7 jjs area=4
    #         I0 0 3 6u
    #         LT0 IN 4 20.5p
    #         LT1 4 3 20.5p
    #         LT2 3 5 20.5p
    #         LT3 5 OUT 20.5p
    #         .ends JTL
        nl = ".subckt JTL IN OUT\n"
        nl += "L1 IN 1 {:.3f}p\n".format(self.Lr/2*1e12)
        nl += "B1 1 0 {} area={:.3f}\n".format(self.jjmodel, self.J1.area)
        nl += "L2 1 2 {:.3f}p\n".format(self.Lr/2*1e12)
        nl += "I1 0 2 {:.3f}u\n".format(self.Ir*1e6)
        nl += "L3 2 3 {:.3f}p\n".format(self.Lr/2*1e12)
        nl += "B2 3 0 {} area={:.3f}\n".format(self.jjmodel, self.J1.area)
        nl += "L4 3 OUT {:.3f}p\n".format(self.Lr/2*1e12)
        nl += ".ends JTL\n"
        
        return nl
    def print_netlist(self):
        print(self.netlist())
        
    def __str__(self):
        return "JTL - Real Units J={:.3f}uA, L={:.3f}pH, I={:.3f}uA".format(self.Jr*1e6, self.Lr*1e12, self.Ir*1e6)
        
    def __repr__(self):
        ret = "JTL:\n"
        ret += "\tJ1 - {}\n".format(str(self.J1))
        ret += "\tL1 - L={:.3f}pH, LLen={:.3f}um\n".format(self.Lr*1e12, self.LLen)
        ret += "\tI1 - I={:.3f}uA".format(self.Ir*1e6)
        return ret

In [383]:
j = JTL(iu=5)
print(j)

JTL - Real Units J=10.000uA, L=98.732pH, I=15.000uA


In [384]:
j

JTL:
	J1 - Ic=10.000uA, d=3.568um, R=9.014Ohm, RLen=9.014um, C=405.000fF
	L1 - L=98.732pH, LLen=135.249um
	I1 - I=15.000uA

In [385]:
j.print_netlist()

.subckt JTL IN OUT
L1 IN 1 49.366p
B1 1 0 jjs area=10.000
L2 1 2 49.366p
I1 0 2 15.000u
L3 2 3 49.366p
B2 3 0 jjs area=10.000
L4 3 OUT 49.366p
.ends JTL



In [393]:
import math

In [395]:
round(5.23452345, 2)

5.23

SyntaxError: invalid syntax (<ipython-input-399-70f614b42f6e>, line 1)